<a href="https://colab.research.google.com/github/blancavazquez/CursoDatosMasivos/blob/master/notebooks/3e_lsh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsqueda del vecino más cercano aproximado mediante funciones _hash_ sensibles a la localidad
En esta libreta se realiza un buscador del vecino más cercano aproximado usando funciones _hash_ sensibles a la localidad (LSH). Especificamente, se define la familia LSH basada  en distribuciones $p$-estables para distancias $\ell_1$ y $\ell_2$.

In [0]:
from os import listdir
from os.path import isfile, join
import struct
import os 

import numpy as np

import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
from skimage import io, transform

Para evaluar el buscador vamos usar el conjunto de vectores SIFT [ANN_SIFT10K](http://corpus-texmex.irisa.fr/) del grupo TEXMEX, el cual descargamos y extraemos.

In [0]:
!wget -q ftp://ftp.irisa.fr/local/texmex/corpus/siftsmall.tar.gz
!tar xvzf siftsmall.tar.gz

siftsmall/
siftsmall/siftsmall_base.fvecs
siftsmall/siftsmall_groundtruth.ivecs
siftsmall/siftsmall_learn.fvecs
siftsmall/siftsmall_query.fvecs


Definimos una función para leer los vectores de un archivo `.fvecs`.

In [0]:
import struct
import os 

def lee_fvecs(ruta):
  with open(ruta, 'rb') as f:
    d = struct.unpack('i', f.read(4))[0]
    n = f.seek(0, os.SEEK_END) // (4 + 4 * d)
    f.seek(0)
    vecs = np.zeros((n, d))
    for i in range(n):
      f.read(4)
      vecs[i] = struct.unpack('f' * d, f.read(d * 4))
  
  return vecs 

Leemos el conjunto de vectores base y consulta.

In [0]:
base = lee_fvecs('siftsmall/siftsmall_base.fvecs')
consultas = lee_fvecs('siftsmall/siftsmall_query.fvecs')

print('Base: {0} Consultas: {1}'.format(base.shape, consultas.shape))

Base: (10000, 128) Consultas: (100, 128)


Definimos una función para leer los vectores más cercanos reales (_groundtruth_) de un archivo `.ivecs`

In [0]:
def lee_ivecs(ruta):
  with open(ruta, 'rb') as f:
    d = struct.unpack('i', f.read(4))[0]
    n = f.seek(0, os.SEEK_END) // (4 + 4 * d)
    f.seek(0)
    vecs = np.zeros((n, d), dtype=np.int)
    for i in range(n):
      f.read(4)
      vecs[i] = struct.unpack('i' * d, f.read(d * 4))
  
  return vecs 

Leemos estos vectores.

In [0]:
gt = lee_ivecs('siftsmall/siftsmall_groundtruth.ivecs')
print('Groundtruth: {0}'.format(gt.shape))

Groundtruth: (100, 100)


Definimos nuestra clase para tabla _hash_ basado en distribuciones $s$-estables para distancias $\ell_1$ y $\ell_2$.

In [0]:
class TablaLpLSH:
  def __init__(self, n_cubetas, t_tupla, dim, width, norma = 'l2'):
    self.n_cubetas = n_cubetas
    self.tabla = [[] for i in range(n_cubetas)]
    self.t_tupla = t_tupla
    self.dim = dim
    self.w = width

    if norma == 'l2':
      self.Amat = np.random.standard_normal((t_tupla, dim))
    elif norma == 'l1':
      self.Amat = np.random.standard_cauchy((t_tupla, dim))

    self.bvec = np.random.uniform(low=0, high=self.w, size=t_tupla)
    self.a = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.b = np.random.randint(0, np.iinfo(np.int32).max, size=self.t_tupla)
    self.primo = 4294967291

  def __repr__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas)]
    return "<TablaHash :%s >" % ('\n'.join(contenido))

  def __str__(self):
    contenido = ['%d::%s' % (i, self.tabla[i]) for i in range(self.n_cubetas) if self.tabla[i]]
    return '\n'.join(contenido)

  def sl(self, x, i):
    return (self.h(x) + i) % self.n_cubetas

  def h(self, x):
    return x % self.primo

  def lphash(self, x):
    prod = np.floor((self.Amat @ x.T + self.bvec) / self.w).astype(int)
    return np.sum(self.a * prod, dtype=np.ulonglong), np.sum(self.b * prod, dtype=np.ulonglong)
     
  def insertar(self, x, ident):
    lph, v2 = self.lphash(x)

    llena = True
    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        self.tabla[cubeta].append(lph)
        self.tabla[cubeta].append([ident])
        llena = False
        break
      elif self.tabla[cubeta][0] == lph:
        self.tabla[cubeta][1].append(ident)
        llena = False
        break

    if llena:
      print('¡Error, tabla llena!')

  def buscar(self, x):
    mh, v2 = self.lphash(x)

    for i in range(self.n_cubetas):
      cubeta = int(self.sl(v2, i))
      if not self.tabla[cubeta]:
        return []
      elif self.tabla[cubeta][0] == mh:
        return self.tabla[cubeta][1]
        
    return []

Instanciamos las tablas.

In [0]:
n_tablas = 100
dim = base.shape[1]
tablas = [TablaLpLSH(2**14, 2, dim, 60.0) for _ in range(n_tablas)]

Insertamos los vectores en cada tabla.

In [0]:
for i,x in enumerate(base):
  for t in range(n_tablas):
    tablas[t].insertar(x, i)

Realizamos la búsqueda de los vectores de consulta y recuperamos los vectores más similares del conjunto base.

In [0]:
vecs = []
for i,q in enumerate(consultas):
  dc = []
  for t in range(n_tablas):
      dc.extend(tablas[t].buscar(q))
  vecs.append(set(dc))

Calculamos la distancia euclidiana entre cada vector de consulta y sus correspondientes vectores recuperados y los ordenamos por distancia.

In [0]:
def distancia_euclidiana(x, y):   
  return np.sqrt(np.sum((x - y)**2))

def fuerza_bruta(ds, qs, fd):
  medidas = np.zeros(ds.shape[0])
  for i,x in enumerate(ds):
    medidas[i] = fd(qs, x)

  return np.sort(medidas), np.argsort(medidas)

dists = []
orden = []
for i,q in enumerate(consultas):
  ld = list(vecs[i])
  if ld:
    m,o = fuerza_bruta(base[ld], q, distancia_euclidiana)
    dists.append(m)
    orden.append([ld[e] for e in o])
  else:
    dists.append([])
    orden.append([])

Extraemos los vecinos más cercanos encontrados por LSH y los reales y los comparamos.

In [0]:
vmc_lsh = [o[0] if o else -1 for o in orden]
vmc_real = [g[0] for g in gt]
correcto = [vmc_lsh[i] == vmc_real[i] for i in range(len(vmc_lsh))]
print('Promedio encontrados = {0}'.format(np.mean(correcto)))

Promedio encontrados = 0.32
